# Importing

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# importing climate and pleasant weather data sets
path = r'C:\Users\Administrator\Desktop\Projects (Carrer foundry)\Data sets'
climate = pd.read_csv(os.path.join(path, 'Dataset-weather-prediction-dataset-processed.csv'))
climate_answers = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [3]:
climate.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


# Data prep

In [9]:
climate = climate.drop(['DATE','MONTH'], axis=1)

In [10]:
climate_answers.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
climate_answers = climate_answers.drop(['DATE'], axis=1)

In [12]:
climate.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [13]:
climate_answers.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# using built in data frame tools to remove the three stations without pleasant data associated 
df = climate.drop(climate.filter(regex='^(GDANSK|ROMA|TOURS)').columns, axis=1)

In [17]:
# checking for the least recorded measurement type
from collections import Counter
measurements = [col.split('_')[-1] for col in df.columns]
measurement_counts = Counter(measurements)
print(measurement_counts)

Counter({'radiation': 15, 'precipitation': 15, 'sunshine': 15, 'mean': 15, 'min': 15, 'max': 15, 'cover': 14, 'humidity': 14, 'pressure': 14, 'speed': 9, 'depth': 6})


In [18]:
# wind speed and snow depth are the two most missed measurements so dropping them
df2 = df.drop(df.filter(regex='(snow_depth|wind_speed)$').columns, axis=1)

In [21]:
measurements = [col.split('_')[-1] for col in df2.columns]
measurement_counts = Counter(measurements)
print(measurement_counts)

Counter({'radiation': 15, 'precipitation': 15, 'sunshine': 15, 'mean': 15, 'min': 15, 'max': 15, 'cover': 14, 'humidity': 14, 'pressure': 14})


In [22]:
df2.shape

(22950, 132)

In [27]:
# checking for what is missing in these three stations 
missing = df2.filter(regex='^(KASSEL|STOCKHOLM|MUNCHENB)')
pd.set_option('display.max_columns', None)
missing.head()

,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max
0,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,5,0.67,0.20,0.10,0.0,6.9,1.1,10.4,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9
1,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.72,0.61,0.30,5.1,6.2,4.2,10.2,5,1.0114,0.05,0.06,0.0,4.0,3.0,5.0
2,0.91,1.0129,0.12,0.60,0.0,6.5,6.0,8.0,6,0.91,0.20,0.30,0.0,5.8,4.0,8.0,5,1.0114,0.05,0.02,0.0,2.4,1.3,4.1
3,0.87,1.0290,0.12,0.00,0.0,5.8,5.2,6.5,6,0.90,0.20,0.01,0.0,3.9,3.2,5.4,5,1.0114,0.05,0.00,0.0,1.2,0.4,2.3
4,0.86,1.0262,0.13,0.71,0.0,5.4,3.7,6.0,5,0.85,0.65,0.96,5.6,1.8,-3.0,6.0,5,1.0114,0.05,1.32,0.0,3.3,0.8,4.3


In [28]:
# Kassel is missing cloud cover, MUNCHENB is missing pressure and Stockholm is missing humidity
# finding the postion of the nearest column for copying data
df2.columns.get_loc('HEATHROW_temp_max') # +1 for kassel cloud cover

53

In [30]:
df2.columns.get_loc('STOCKHOLM_cloud_cover') # +2 for stockholm humidity

115

In [31]:
df2.columns.get_loc('MUNCHENB_humidity') # +2 for muchenb pressure 

90

In [37]:
df3 = df2.copy()

In [38]:
# The data for these new columns is copied from other existing columns:
# Kassel_cloud_cover with Dusseldorf_cloud_cover
# Stockholm_humidity with Oslo_humidity
# Munchenb_pressure with Basel_pressure

df3.insert(54,'KASSEL_cloud_cover', df3['DUSSELDORF_cloud_cover'])
df3.insert(117, 'STOCKHOLM_humidity', df3['OSLO_humidity'])
df3.insert(92,'MUNCHENB_pressure',df3['BASEL_pressure'])

In [39]:
df3.columns.tolist()

['BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_temp_mean',


In [40]:
measurements = [col.split('_')[-1] for col in df3.columns]
measurement_counts = Counter(measurements)
print(measurement_counts)

Counter({'cover': 15, 'humidity': 15, 'pressure': 15, 'radiation': 15, 'precipitation': 15, 'sunshine': 15, 'mean': 15, 'min': 15, 'max': 15})


In [41]:
df3.shape

(22950, 135)

In [42]:
climate_answers.shape

(22950, 15)

# Exporting clean Data

In [44]:
df3.to_csv(os.path.join(path, 'Climate_Cleaned.csv'))
climate_answers.to_csv(os.path.join(path, 'Pleasant_Day_Cleaned.csv'))

# Keras modeling (CNN)

In [45]:
# creating the arrays for analysis
X = df3.copy()
y = climate_answers.copy()
X = np.array(X)
y = np.array(y)
X = X.reshape(-1,15,9)

In [46]:
X.shape

(22950, 15, 9)

In [47]:
y.shape

(22950, 15)

In [48]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [50]:
X_train.shape

(17212, 15, 9)

# Begin Iterating on CNN hyperparameters

In [51]:
epochs = 15
batch_size = 8
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [52]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 16)         │           304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14, 16)         │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,271 (8.87 KB)

 Trainable params: 2,271 (8.87 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 3s - 1ms/step - accuracy: 0.6434 - loss: 64602648.0000
Epoch 2/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 76553576.0000
Epoch 3/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 89172976.0000
Epoch 4/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 103759424.0000
Epoch 5/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 120539576.0000
Epoch 6/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 138121280.0000
Epoch 7/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 157039904.0000
Epoch 8/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 177739920.0000
Epoch 9/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 201718736.0000
Epoch 10/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 225336048.0000
Epoch 11/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 251555456.0000
Epoch 12/15
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 280882912.0000
Epoch 13/15
2152/2152 - 2s - 1ms/step - accuracy: 0.

In [55]:
epochs = 30
batch_size = 8
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [56]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
2152/2152 - 3s - 1ms/step - accuracy: 0.6261 - loss: 14807.5410
Epoch 2/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 148475.5312
Epoch 3/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 453071.6250
Epoch 4/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 1015357.0625
Epoch 5/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 1864835.0000
Epoch 6/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 2979234.0000
Epoch 7/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 4513962.5000
Epoch 8/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 6336583.5000
Epoch 9/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 8718454.0000
Epoch 10/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 11550408.0000
Epoch 11/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 15046659.0000
Epoch 12/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 18989372.0000
Epoch 13/30
2152/2152 - 2s - 1ms/step - accuracy: 0.6434 - loss: 23723562.

In [57]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [58]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.2360 - loss: 26.8052
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2607 - loss: 28.0306
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2613 - loss: 28.0278
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2611 - loss: 28.0278
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2610 - loss: 28.0287
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2609 - loss: 28.0269
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2864 - loss: 29.6226
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.3118 - loss: 30.5384
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.5335 - loss: 28.0316
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6317 - loss: 26.8994
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6323 - loss: 27.1644
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6340 - loss: 27.1560
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6348 - loss: 27.0221
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6354 - l

In [59]:
epochs = 15
batch_size = 16
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [60]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
1076/1076 - 2s - 2ms/step - accuracy: 0.1968 - loss: 29.8066
Epoch 2/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1170 - loss: 30.6313
Epoch 3/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1144 - loss: 30.6339
Epoch 4/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1121 - loss: 30.6227
Epoch 5/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1113 - loss: 30.6189
Epoch 6/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 7/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 8/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 9/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 10/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 11/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 12/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 13/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - loss: 30.6189
Epoch 14/15
1076/1076 - 1s - 1ms/step - accuracy: 0.1114 - l

In [61]:
epochs = 15
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [62]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
1076/1076 - 2s - 2ms/step - accuracy: 0.0576 - loss: 27.7963
Epoch 2/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2358 - loss: 21.0694
Epoch 3/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2666 - loss: 20.2122
Epoch 4/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2668 - loss: 20.2413
Epoch 5/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2670 - loss: 20.2441
Epoch 6/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2672 - loss: 20.2497
Epoch 7/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2676 - loss: 20.2572
Epoch 8/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2686 - loss: 20.2656
Epoch 9/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2692 - loss: 20.2750
Epoch 10/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2695 - loss: 20.2834
Epoch 11/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2697 - loss: 20.2909
Epoch 12/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2697 - loss: 20.2909
Epoch 13/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2697 - loss: 20.2909
Epoch 14/15
1076/1076 - 1s - 1ms/step - accuracy: 0.2697 - l

In [63]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.3658 - loss: 24.9991
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.5660 - loss: 25.1250
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.5706 - loss: 25.1304
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.5733 - loss: 25.0705
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.5213 - loss: 23.9683
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4937 - loss: 23.2575
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4939 - loss: 23.2575
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4940 - loss: 23.2575
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4941 - loss: 23.2575
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4941 - loss: 23.2576
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4945 - loss: 23.2575
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4947 - loss: 23.2575
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4949 - loss: 23.2575
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4953 - l

In [65]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [66]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1039 - loss: 6130.0713
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1258 - loss: 63679.5742
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1301 - loss: 207601.3438
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1335 - loss: 476720.5000
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1354 - loss: 870771.2500
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1345 - loss: 1418664.2500
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1386 - loss: 2107267.0000
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1350 - loss: 2988271.0000
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1369 - loss: 4073439.2500
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1350 - loss: 5332136.0000
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1365 - loss: 6774249.0000
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1355 - loss: 8415929.0000
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1344 - loss: 10304874.0000
Ep

In [67]:
epochs = 10
batch_size = 8
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [68]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
2152/2152 - 4s - 2ms/step - accuracy: 0.1104 - loss: 188371.2031
Epoch 2/10
2152/2152 - 3s - 1ms/step - accuracy: 0.1373 - loss: 1906187.0000
Epoch 3/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1346 - loss: 6131111.0000
Epoch 4/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1362 - loss: 13508194.0000
Epoch 5/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1364 - loss: 24831538.0000
Epoch 6/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1319 - loss: 40508048.0000
Epoch 7/10
2152/2152 - 3s - 1ms/step - accuracy: 0.1344 - loss: 60945088.0000
Epoch 8/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1337 - loss: 87247912.0000
Epoch 9/10
2152/2152 - 3s - 1ms/step - accuracy: 0.1348 - loss: 121237112.0000
Epoch 10/10
2152/2152 - 2s - 1ms/step - accuracy: 0.1372 - loss: 159318720.0000


In [69]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [70]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1419 - loss: 23.1101
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.4134 - loss: nan
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/30
1076/1076 - 1s - 1ms/step -

In [71]:
epochs = 30
batch_size = 32
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
538/538 - 1s - 3ms/step - accuracy: 0.0929 - loss: 20.7219
Epoch 2/30
538/538 - 1s - 1ms/step - accuracy: 0.1354 - loss: 16.7254
Epoch 3/30
538/538 - 1s - 1ms/step - accuracy: 0.2677 - loss: nan
Epoch 4/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/30
538/538 - 1s - 1ms/step - accuracy: 0.6440 - loss: 

In [73]:
epochs = 10
batch_size = 16
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [74]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
1076/1076 - 2s - 2ms/step - accuracy: 0.1428 - loss: 17.8219
Epoch 2/10
1076/1076 - 1s - 1ms/step - accuracy: 0.1941 - loss: 12.5746
Epoch 3/10
1076/1076 - 1s - 1ms/step - accuracy: 0.5554 - loss: nan
Epoch 4/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan


In [75]:
epochs = 30
batch_size = 16
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [76]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6212 - loss: 34133.2930
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6426 - loss: 376560.5000
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6430 - loss: 1217257.7500
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6431 - loss: 2692944.5000
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6433 - loss: 4843956.5000
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6434 - loss: 7594516.5000
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6435 - loss: 11253440.0000
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 15778887.0000
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 21047918.0000
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6436 - loss: 27448070.0000
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 35386164.0000
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 43608696.0000
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6436 - loss: 53038

In [83]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [84]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0016 - loss: 23.1184
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0020 - loss: 28.6535
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0023 - loss: 28.7572
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0021 - loss: 28.7629
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0021 - loss: 28.7657
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0020 - loss: 28.7676
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0020 - loss: 28.7741
Epoch 8/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0060 - loss: 27.3892
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0304 - loss: 19.2580
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0318 - loss: 18.8085
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0328 - loss: 18.6830
Epoch 12/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0332 - loss: 18.6268
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0335 - loss: 18.7748
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0336 - l

In [86]:
epochs = 10
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [87]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
1076/1076 - 2s - 2ms/step - accuracy: 0.6209 - loss: 7597.9595
Epoch 2/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6435 - loss: 85819.2109
Epoch 3/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 281249.7812
Epoch 4/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6438 - loss: 619116.2500
Epoch 5/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 1107989.8750
Epoch 6/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 1784685.7500
Epoch 7/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6436 - loss: 2574670.0000
Epoch 8/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6436 - loss: 3594610.2500
Epoch 9/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6437 - loss: 4819203.0000
Epoch 10/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6438 - loss: 6274919.0000


In [90]:
epochs = 10
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [91]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
1076/1076 - 2s - 2ms/step - accuracy: 0.3638 - loss: nan
Epoch 2/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/10
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: nan


# Confusion Matrices

In [92]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [93]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

## Final Matrix

In [94]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1


## Starting Matrix

In [95]:
epochs = 15
batch_size = 8
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [96]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

In [97]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step
Pred        BASEL  VALENTIA
True                       
BASEL        3680         2
BELGRADE     1092         0
BUDAPEST      214         0
DEBILT         82         0
DUSSELDORF     29         0
HEATHROW       82         0
KASSEL         11         0
LJUBLJANA      61         0
MAASTRICHT      9         0
MADRID        458         0
MUNCHENB        8         0
OSLO            5         0
STOCKHOLM       4         0
VALENTIA        1         0


# Tanh test

In [98]:
epochs = 15
batch_size = 8
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [99]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

In [100]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step
Pred        BUDAPEST  DEBILT  HEATHROW  MAASTRICHT  MADRID  MUNCHENB  VALENTIA
True                                                                          
BASEL           3416      93         1           3       1        80        88
BELGRADE        1090       1         0           0       0         1         0
BUDAPEST         214       0         0           0       0         0         0
DEBILT            82       0         0           0       0         0         0
DUSSELDORF        29       0         0           0       0         0         0
HEATHROW          82       0         0           0       0         0         0
KASSEL            11       0         0           0       0         0         0
LJUBLJANA         61       0         0           0       0         0         0
MAASTRICHT         9       0         0           0       0         0         0
MADRID           458       0         0           0       0         0         0
MUNCHENB 

In [101]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [102]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)

In [103]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step
Pred        BASEL  BELGRADE  DEBILT  DUSSELDORF  OSLO  SONNBLICK  VALENTIA
True                                                                      
BASEL        2507       598      21           3    20        515        18
BELGRADE      697        16      14           0     0        365         0
BUDAPEST      133         6       5           0     0         70         0
DEBILT         76         2       0           0     0          4         0
DUSSELDORF     24         0       1           0     0          4         0
HEATHROW       53         9       9           0     0         11         0
KASSEL         11         0       0           0     0          0         0
LJUBLJANA      35         2       4           0     0         20         0
MAASTRICHT      9         0       0           0     0          0         0
MADRID        208        66      11           0     2        171         0
MUNCHENB        4         0       0           0     0    